In [1]:
import numpy as np
import geopandas as gpd
import dill
import matplotlib.pyplot as plt
import numpy.ma as ma
from discretize import TensorMesh
from ipywidgets import widgets, interact

In [2]:
output_dict = dill.load(open("../metric_maps/output_metric_maps.pik", "rb"))

In [3]:
fc_thresholds = [0.2, 0.5, 0.8]
fc = output_dict['fc']

In [4]:
thres = fc_thresholds[0]
path_length_norm = output_dict['path_length_norm'][thres]
path_to_no_flow = output_dict['path_to_no_flow'][thres]

thres_path_length_norm = 50.
thres_path_to_no_flow = 10.
fc_threshold = 0.8

def get_summary_metric_map(fc_threshold, thres_path_length_norm, path_to_no_flow):
    inds_all = np.logical_and(fc.data>fc_threshold, path_length_norm.data<thres_path_length_norm, path_to_no_flow.data>thres_path_to_no_flow)
    summary_metric_up = inds_all.astype(float)
    summary_metric_up = ma.masked_array(data=summary_metric_up, mask=fc.mask)
    return summary_metric_up

In [5]:
mesh_orig = TensorMesh.read_UBC("../metric_maps/mesh_orig.msh")

In [6]:
from matplotlib.colors import LinearSegmentedColormap

colors = [(0, 0.5, 0),    # Green
          (1, 1, 1)]    # White
positions = [0, 1]
green_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

colors = [(1, 1, 1),    # Green
          (0, 0.5, 0)]    # White
positions = [0, 1]
green_r_map = LinearSegmentedColormap.from_list("green_white", list(zip(positions, colors)))

# Extract RGB values from the predefined RdBu colormap
cmap = plt.get_cmap('RdBu')

# Create a list of colors by sampling the colormap
colors = [cmap(i) for i in range(cmap.N)]

# Create the custom colormap
custom_rdbu = LinearSegmentedColormap.from_list("custom_rdbu", np.array(colors)[20:-20][::-1])

import matplotlib.colors
binary_cmap = matplotlib.colors.ListedColormap(['white', 'green'])

In [7]:
import rasterio
from rasterio.plot import show
src = rasterio.open("../data/cv_terrain.tiff")
gdf_rivers = gpd.read_file("../data/shp/cv_rivers.geojson")
gdf_rivers_3310 = gdf_rivers.to_crs(3310)
gdf_rho_salinity = gpd.read_file("../data/shp/rho_salinity_boundary/rho_salinity_boundary.shp")
gdf_subbasin_cv = gpd.read_file("../data/shp/subbasins/subbasins_cv.shp")

In [8]:
from matplotlib.colors import LogNorm

In [11]:
options_path_length_norm = [1, 2, 5, 20, 100]
options_path_to_no_flow = [1, 5, 10, 20, 40]

def foo(metric_map, thres_fc, show_brackish_region, show_subbasins, show_rivers, show_terrain):

    fig, ax = plt.subplots(1,1, figsize=(5, 8))
    if show_terrain:
        show(src, ax=ax, alpha=1)
    
    path_length_norm = output_dict['path_length_norm'][thres_fc]
    path_to_no_flow = output_dict['path_to_no_flow'][thres_fc]

    if metric_map == 'fcd':
        cmap = custom_rdbu
        values = fc.copy() * 100
        norm = None
        vmin, vmax = 0, 100
        clim = (vmin, vmax)
        norm = None    
        label = "Averaged FCD (%)"
    elif metric_map == 'path_length_norm':
        cmap = green_map
        values = path_length_norm.copy()
        vmin, vmax = 1, 1000
        clim = None
        norm = LogNorm(vmin=vmin, vmax=vmax)
        label = "Normalized path length"
    elif metric_map == 'path_to_no_flow':
        cmap = green_r_map
        values = path_to_no_flow.copy()
        vmin, vmax = 0, 20
        clim = (vmin, vmax)
        norm = None
        label = "Depth to shallowest no-flow or base (m)"
    elif metric_map == 'summary':        
        cmap = binary_cmap
        values = get_summary_metric_map(thres_fc, thres_path_length_norm, path_to_no_flow)
        vmin, vmax = 0, 1
        clim = (vmin, vmax)
        norm = None
        label = "Summary"
    
    out = mesh_orig.plot_image(values, ax=ax, clim=clim, pcolor_opts={'cmap':cmap, 'norm':norm})

    if show_brackish_region:
        gdf_rho_salinity.plot(ax=ax, legend=True, color='grey')
    if show_rivers:
        gdf_rivers_3310.plot(color='blue', linewidth=0.3, ax=ax, alpha=0.8, label='Rivers & streams')
    if show_subbasins:
        gdf_subbasin_cv.boundary.plot(color='k', linewidth=0.3, ax=ax, label='Subbasin boundaries')
        
    cb = plt.colorbar(out[0], orientation='horizontal', fraction=0.02, pad=0.02, ax=ax)
    if metric_map == 'summary':
        cb.set_ticks([0.25, 0.75])
        cb.set_ticklabels(["Unsuitable", "Suitable"])
        cb.set_label(" ")        
    else:
        cb.set_label(label)    
    
    # ax.set_title(title)
    ax.set_aspect(1)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_yticks([])  
    ax.set_xlabel("")
    ax.set_ylabel("")    
    # ax.axis('off')
    ax.set_aspect(1)
    ax.set_facecolor('lightgrey')
    plt.tight_layout()

In [12]:
interact(
    foo,
    metric_map=widgets.Select(options=['fcd', 'path_length_norm', 'path_to_no_flow', 'summary']),
    thres_fc=widgets.SelectionSlider(options=[('20%', 0.2), ('50%', 0.5), ('80%', 0.8)], value=0.5), 
    # thres_path_length_norm=widgets.SelectionSlider(options=options_path_length_norm, value=20),
    # thres_path_to_no_flow=widgets.SelectionSlider(options=options_path_to_no_flow, value=10),
    show_brackish_region=widgets.Checkbox(value=True),
    show_subbasins=widgets.Checkbox(value=False),
    show_rivers=widgets.Checkbox(value=False),
    show_terrain=widgets.Checkbox(value=False)
)

interactive(children=(Select(description='metric_map', options=('fcd', 'path_length_norm', 'path_to_no_flow', …

<function __main__.foo(metric_map, thres_fc, show_brackish_region, show_subbasins, show_rivers, show_terrain)>